In [1]:
def get_library_path()->str:

    cwd = os.getcwd()
    lst = cwd.split(os.sep)[:-1]

    path_lib = f'{os.sep}'

    for elem in lst:
        path_lib = os.path.join(path_lib, elem)

    return path_lib

# imports

import os
import sys

path_to_lib = get_library_path()

sys.path.append(path_to_lib)
import luxgiant_clinical.ThreeCatAnalysis as thr

import pandas as pd

In [2]:
# load data

folder_path = os.path.join(path_to_lib, 'data')

df = pd.read_csv(
    os.path.join(folder_path, 'cleaned_file.csv'), 
    low_memory=False
)

In [3]:
# filter data to keep only patients with age at onset information

mask_patients= (df['Status']=='Patient')
mask_subtype = ~df['subtype'].isnull()
mask_off     = (df['on_off']=='OFF: Off is the typical functional state when patien ts have a poor response in spite of taking medications.')

df_cases = df[mask_patients & mask_off & mask_subtype].reset_index(drop=True)
del df

df_cases.shape

(1815, 716)

In [4]:
df_off = df_cases.reset_index(drop=True)

df_off['Stage I-III']= df_off['hystage'].map({'Not severe':1, 'Severe':0})
df_off['Stage IV-V'] = df_off['hystage'].map({'Not severe':0, 'Severe':1})
df_off['UPDRS I-OFF']= df_off['updrs_1_total_score'].copy()
df_off['UPDRS II-OFF']= df_off['updrs_part_ii_total_score'].copy()
df_off['UPDRS III-OFF']= df_off['updrs_part_iii_total_score'].copy()


variables = ['participant_id', 'subtype', 'Stage I-III', 'Stage IV-V', 'UPDRS I-OFF', 'UPDRS II-OFF', 'UPDRS III-OFF']

stats_meas2 = {
    'n': ['Stage I-III', 'Stage IV-V'],
    'median' : ['UPDRS I-OFF', 'UPDRS II-OFF', 'UPDRS III-OFF']
}
groups = ['Tremor Dominant', 'Indeterminate', 'Postural instability and gait difficulty']

# keep only variables for analysis

df_off = df_off[variables].copy()

In [5]:
summary1 = [
    thr.report_proportion(
        data_df    =df_off,
        variables  =stats_meas2['n'],
        groups     =groups,
        grouping_by='subtype',
        subheader  ='H & Y OFF'
        ),
    thr.report_median_iqr(
        data_df    =df_off,
        variables  =stats_meas2['median'],
        groups     =groups,
        grouping_by='subtype'
    )
]

In [6]:
variables_dict = {
    "total_score_for_moca": "MOCA Score",
    "total_score_for_bdi" : "BDI Score" 
}
variables_1 = ['participant_id', 'subtype'] + list(variables_dict.keys())

# statistical measures
stats_meas3 = {
    'mean'  : ["MOCA Score"],
    'median': ["BDI Score"]
}

# keep only variables for analysis

df_cases_1 = df_cases[variables_1].copy()
df_cases_1 = df_cases_1.rename(columns=variables_dict)

In [7]:
summary2 = [ 
    thr.report_mean_std(
        data_df    =df_cases_1,
        variables  =stats_meas3['mean'],
        groups     =groups,
        grouping_by='subtype'
    ),
    thr.report_median_iqr(
        data_df    =df_cases_1,
        variables  =stats_meas3['median'],
        groups     =groups,
        grouping_by='subtype'
    )
]

In [8]:
df_15 = pd.concat(
    summary1 + summary2, axis=0, ignore_index=True
)

In [9]:
df_off_1 = df_off[~(df_off['subtype']=='Indeterminate')].reset_index(drop=True)
df_cases_2 = df_cases_1[~(df_cases_1['subtype']=='Indeterminate')].reset_index(drop=True)

In [10]:
summary3 = [
    thr.bonferroni_median_iqr(
        data_df      =df_off_1, 
        variables    =stats_meas2['median'], 
        groups       =['Tremor Dominant',  'Postural instability and gait difficulty'],
        grouping_by  ='subtype',
        correc_factor=3
    ),
    thr.bonferroni_proportions(
        data_df      =df_off_1, 
        variables    =stats_meas2['n'], 
        groups       =['Tremor Dominant',  'Postural instability and gait difficulty'],
        grouping_by  ='subtype',
        subheader  ='H & Y OFF',
        correc_factor=3
    ),
    thr.bonferroni_mean_std(
        data_df      =df_cases_2, 
        variables    =stats_meas3['mean'], 
        groups       =['Tremor Dominant',  'Postural instability and gait difficulty'],
        grouping_by  ='subtype',
        correc_factor=3
    ),
    thr.bonferroni_mean_std(
        data_df      =df_cases_2, 
        variables    =stats_meas3['median'], 
        groups       =['Tremor Dominant',  'Postural instability and gait difficulty'],
        grouping_by  ='subtype',
        correc_factor=3
    )
]

In [11]:
df_151 = pd.concat(
    summary3, axis=0,ignore_index=True
)

In [12]:
df_15 = thr.final_formatter(
    overall_df =df_15, 
    adjusted_df=[df_151], 
    groups     =['Tremor Dominant', 'Indeterminate', 'Postural instability and gait difficulty']
)

In [13]:
df_15.to_csv(os.path.join(folder_path, 'Table_15.csv'), index=False)
df_15

,Variable,Statistical Measure,Tremor Dominant,Indeterminate,Postural instability and gait difficulty,Total,p-value,Adjusted p-value,Available Samples for Analysis
0,H & Y OFF,,,,,,,,
1,Stage I-III,n (%),590 (98.3),339 (96.3),771 (90.7),1700 (94.3),p<0.001,p<0.001,1802
2,Stage IV-V,n (%),10 (1.7),13 (3.7),79 (9.3),102 (5.7),p<0.001,p<0.001,1802
3,UPDRS I-OFF,median (IQR),0.0 (0.0 - 1.0),1.0 (0.0 - 2.0),1.0 (0.0 - 3.0),1.0 (0.0 - 2.0),p<0.001,p<0.001,1795
4,UPDRS II-OFF,median (IQR),9.0 (6.0 - 11.0),10.0 (8.0 - 15.0),12.0 (9.0 - 16.0),10.0 (7.0 - 15.0),p<0.001,p<0.001,1809
5,UPDRS III-OFF,median (IQR),35.0 (25.0 - 46.0),39.0 (31.0 - 46.0),39.0 (28.0 - 48.0),38.0 (27.0 - 47.0),0.0012,0.0067,1476
6,MOCA Score,mean (SD),25.3 (6.1),24.8 (6.0),24.2 (5.8),24.7 (6.0),0.0054,0.0039,1533
7,BDI Score,median (IQR),8.0 (4.0 - 12.0),9.0 (5.0 - 13.0),9.0 (6.0 - 14.0),9.0 (5.0 - 13.0),p<0.001,p<0.001,1646
